In [13]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import os
from dotenv import load_dotenv
import time

In [14]:
load_dotenv()
REED_USER = os.getenv('REED_USER')

JOB_TITLES = ["data science", "data engineer", "data analyst"]

In [21]:
from typing import Any


class ReedAPI:
    def __init__(self, api_key: str):
        self.user = api_key
        self.password = ""
        self.base_url = "https://www.reed.co.uk/api/1.0/search/"
        self.page_size = 50

    def get_all_jobs(self, job_title: str) -> list[dict]:
        jobs, total_results = self.get_jobs_partial(job_title, self.page_size, 0)
        for page in range(1, total_results // self.page_size + 1):
            time.sleep(1) # TODO Check the API rate limit
            jobs.extend(self.get_jobs_partial(job_title, self.page_size, page)[0])

            if page > 1: # TODO Remove this, it's just for testing
                break
        return jobs
    
    def get_jobs_partial(self, job_title: str, count: int, page: int) -> tuple[list[dict], int]:
        print(f"Getting jobs for {job_title} page {page}")
        params = {
            "keywords" : job_title,
            "resultsToTake" : count,
            "resultsToSkip" : page * count
        }
        auth = HTTPBasicAuth(self.user, self.password)
        response = requests.get(self.base_url, auth=auth, params=params)
        if response.status_code != 200:
            raise Exception("Failed to get jobs")
        response_json = response.json()
        print(f"Got {len(response_json.get('results'))} jobs for {job_title} page {page}. Total results: {response_json.get('totalResults')}")
        return (response_json.get("results"), int(response_json.get("totalResults")))
    
def load(dict_list: list[dict]) -> pd.DataFrame:
    df = pd.DataFrame(dict_list)
    df_selected = df[['jobId', 'employerId','employerName', 'jobTitle', 'locationName', 'minimumSalary', 'maximumSalary', 'currency', 'date', 'applications', 'jobUrl']].copy()
    df_selected['date'] = pd.to_datetime(df_selected['date'], format="%d/%m/%Y", errors='coerce')
    return df_selected
    

In [19]:
reed_api = ReedAPI(REED_USER)
datascience_jobs = reed_api.get_all_jobs("data science")
print(len(datascience_jobs))

Getting jobs for data science page 0
Got 50 jobs for data science page 0. Total results: 1428
Getting jobs for data science page 1
Got 50 jobs for data science page 1. Total results: 1428
Getting jobs for data science page 2
Got 50 jobs for data science page 2. Total results: 1428
150


In [22]:
jobs_df = load(datascience_jobs)
print(jobs_df.shape)
jobs_df.head()

(150, 11)


,jobId,employerId,employerName,jobTitle,locationName,minimumSalary,maximumSalary,currency,date,applications,jobUrl
0,50980134,389257,Morgan McKinley,Data Science Manager,London,80000.0,80000.0,GBP,2023-08-08,11,https://www.reed.co.uk/jobs/data-science-manag...
1,50938687,383872,Rutherford Briant,Data Science Lead,CO49YA,90000.0,110000.0,GBP,2023-08-01,23,https://www.reed.co.uk/jobs/data-science-lead/...
2,50986900,575264,Reed,Data Science Lead,M405BP,735.0,735.0,GBP,2023-08-08,12,https://www.reed.co.uk/jobs/data-science-lead/...
3,50963399,331522,Harnham - Data & Analytics Recruitment,Data Science Manager,London,75000.0,85000.0,GBP,2023-08-04,12,https://www.reed.co.uk/jobs/data-science-manag...
4,51055661,331522,Harnham - Data & Analytics Recruitment,Director of Data Science,London,100000.0,115000.0,GBP,2023-08-18,5,https://www.reed.co.uk/jobs/director-of-data-s...


In [28]:
transformed_df = jobs_df.groupby('locationName').agg(
    {
        'jobId': 'count',
        'minimumSalary': 'mean',
        'maximumSalary': 'mean',
        'date': 'max'
    }).reset_index()
transformed_df.columns = ['location', 'job_count', 'avg_min_salary', 'avg_max_salary', 'last_job_post']
transformed_df["avg_max_salary"] = transformed_df["avg_max_salary"].round(0)
transformed_df["avg_min_salary"] = transformed_df["avg_min_salary"].round(0)
transformed_df.sort_values('job_count', ascending=False).head(10)

,location,job_count,avg_min_salary,avg_max_salary,last_job_post
43,London,45,78984.0,94267.0,2023-08-23
45,Manchester,5,56667.0,66667.0,2023-07-31
11,Cambridge,5,45000.0,55000.0,2023-08-15
41,Leeds,4,73750.0,86250.0,2023-08-22
47,N156EB,4,32157.0,50935.0,2023-08-15
48,N165RS,4,32247.0,48452.0,2023-08-16
46,Milton Keynes,3,20300.0,21400.0,2023-08-14
64,Telford,3,19853.0,22280.0,2023-07-21
8,Bristol,3,25060.0,32582.0,2023-08-23
26,Edinburgh,3,30300.0,32825.0,2023-08-23
